# Домашнее задание № 3. Исправление опечаток

In [15]:
import re
from collections import Counter
from string import punctuation

import textdistance

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

### Данные
Корпус из Wikipedia в качестве словаря правильных слов

In [6]:
# считываем корпус
corpus = open('wiki_data.txt', encoding='utf8').read()
# создаем частотный словарь
WORDS = Counter(re.findall('\w+', corpus.lower()))

Корпус с соревнования по исправлению опечаток [Dialog Evaluation 2016](http://www.dialog-21.ru/evaluation/2016/spelling_correction/) для оценки качества алгоритма исправления ошибок

In [7]:
# предложения с ошибками
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
# предложения без ошибок
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.



### Решение

Для поиска наиболее вероятного кандидата будем использовать функцию для рассчета вероятности встретить слово word в словаре WORDS:

In [ ]:
def P(word, N=sum(WORDS.values())): 
    """ Вероятность встретить слово в словаре """
    
    return WORDS[word] / N

Единственное изменение, которое потребуется внести в код из семинара — возвращаемое значение метода get_closest_hybrid_match. Вместо списка из кандидатов на исправление будем возвращать только одно слово — для которого наибольшая вероятность встретиться в корпусе

In [37]:
# оригинальный код для методов ниже лежит тут: 
# https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/spelling/Spellchecking.ipynb

def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    # изменяем в исходном коде эту строчку:
    # return closest
    # из полученных кандидитов выбираем наиболее вероятный вариант
    return list(max(closest, key=P))[0] # вытаскиваем только само слово

In [39]:
get_closest_hybrid_match('метвежёнок', X, vec)

'медвежонок'

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

### Реализация алгоритма SymSpell

Отредактируем алгоритм [Норвига](https://norvig.com/spell-correct.html), оставив только операцию удаления

In [40]:
def get_deletes1(word):
    """ Поочередно удаляем из слова 1 букву """
    
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    
    return set(deletes)

def get_deletes_vocab():
    """ Создаем словарь удалений """
    
    deletes_vocab = {}
    for w in WORDS:
        deletes_vocab[w] = get_deletes1(w)
    
    return deletes_vocab

# создаем словарь удалений и сохраняем его как константу
VOCAB = get_deletes_vocab()


def get_candidates(word): 
    """ Генерируем кандидатов на исправление """
    
    candidates = []
    # генерируем все варианты удалений для слова
    words = get_deletes1(word)
    # ищем совпадения в словаре удалений
    for key in VOCAB.keys():
        if len(words.intersection(VOCAB[key])) > 0:
            candidates.append(key)
                                
    return set(candidates)

def make_correction(word): 
    """ Находим наиболее вероятное похожее слово """
    
    if word in WORDS:
        return word
    else:
        candidates = get_candidates(word)
        if len(candidates) == 1: 
            return list(candidates)[0]
        elif len(candidates) > 1:
            return list(max(get_candidates(word), key=P))[0]
        # если исправления не найдены, возвращаем исходное слово
        else:
            return word


Структура словаря VOCAB: ключ — правильное слово, значение — множество всех удалений (выбрасывается только одна буква)

In [41]:
VOCAB['медвежонок']

{'едвежонок',
 'мдвежонок',
 'мевежонок',
 'медвежнок',
 'медвежонк',
 'медвежоно',
 'медвежоок',
 'медвеонок',
 'медвжонок',
 'медежонок'}

Попробуем исправить ошибки

In [42]:
make_correction('медвежёнок')

'медвежонок'

In [43]:
make_correction('здрафствуйте')

'здравствуйте'

Алгоритм не сработает, если ошибка в слове заключается в пропуске буквы

In [44]:
make_correction('здраствуйте')

'здраствуйте'

### Оценка качества алгоритма

Оценим качество алгоритма, прогнав его на корпусе с соревнования по исправлению опечаток

Метрики для оценки качества алгоритма:

— процент правильных слов;  
— процент исправленных ошибок  
— процент ошибочно исправленных правильных слов

In [46]:
# код для рассчета метрик взят отсюда
# https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/spelling/Spellchecking.ipynb

# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуацию на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [47]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], make_correction(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [48]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8471235617808904
0.13043478260869565
0.04685884920179166


### Вывод

Алгоритм SymSpell позволяет исправлять определенные типы ошибок, однако предложенная в данной тетрадке реализация работает хуже, чем алгоритм Норвига, реализованный на [семинаре](https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/spelling/Spellchecking.ipynb). Возможно, оптимизация алгоритма и добавление дополнительной операции удаления позволит улучшить качество SymSpell.

Процент исправленных ошибок для SymSpell (с одной операцией удаления): 13%

Процент исправленных ошибок алгоритмом Норвига на этом же корпусе: 51%